In [1]:
import warnings

import os
import torch
import pickle
import numpy as np
import os.path as osp
from tqdm import tqdm
from functools import reduce
import matplotlib.pyplot as plt

from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from src.datasets import DatasetBuilder
from src.models.classifiers import SimpleCNNtorch
from src.models.lightning_wrappers import ClassifierLightningWrapper 

from src.utils.generic_utils import seed_everything, get_config, load_model_weights
from src.utils.generic_utils import evaluate_classification_model

d:\PycharmProjects\CF-Robustness-Benchmark\venv\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
seed_everything()
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
config_path = r"D:\PycharmProjects\CF-Robustness-Benchmark\configs\train_classifier_mnist.yaml"
config = get_config(config_path)

In [4]:
ds_builder = DatasetBuilder(config)
ds_builder.setup()
train_loader, val_loader, test_loader = ds_builder.get_dataloaders()

In [5]:
baseline_classifier = SimpleCNNtorch(**config.classifier.args,
                                     img_size=config.data.img_size)
load_model_weights(baseline_classifier, weights_path=config.classifier.checkpoints_path)
evaluate_classification_model(baseline_classifier, test_loader, config.data.num_classes)

Accuracy for the test dataset: 100.000%


In [6]:
experiment_dir = r"D:\PycharmProjects\CF-Robustness-Benchmark\notebooks\experiments"
experiment_name = "mnist_classification"
experiment_version = "binary"  if config.data.num_classes == 2 else "multiclass"
checkpints_dir = osp.join(experiment_dir, experiment_name, experiment_version, 'checkpoints', 'mc_1_8')
class_names = ds_builder.class_encodings
classes4fname = ("_").join([str(i) for i in class_names.values()]) if config.data.num_classes == 2 else ""

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.00,
    patience=3,
    verbose=False,
    mode="min",
)
tb_logger = TensorBoardLogger(save_dir=experiment_dir, name=experiment_name, version=experiment_version)

In [ ]:
from random import randint
from torchmetrics import Accuracy


seed_list = [randint(1000, 5000) for _ in range(10)]
num_classes = config.data.num_classes
baseline_accuracy = 1.0

for seed in tqdm(seed_list):
    seed_everything(seed)

    cnn_wi = SimpleCNNtorch(**config.classifier.args,
                            img_size=config.data.img_size)
    cnn_wrapper = ClassifierLightningWrapper(config, cnn_wi)
    chekpoint_callback = ModelCheckpoint(
        monitor="val_accuracy",
        dirpath=checkpints_dir,
        filename=f"mnist_{classes4fname}_{seed}",
        save_top_k=1,
        mode="max",
        save_weights_only=True,
    )
    chekpoint_callback.FILE_EXTENSION = '.pth'
    trainer = Trainer(
        log_every_n_steps=10,
        max_epochs=15,
        enable_checkpointing=True,
        callbacks=[early_stop_callback, chekpoint_callback],
        logger=tb_logger
    )
    trainer.fit(model=cnn_wrapper, 
            train_dataloaders=train_loader, 
            val_dataloaders=val_loader)
    
    # Check predictive power of the model
    calc_metric = Accuracy(task="binary" if num_classes==2 else "multiclass", num_classes=num_classes)

    accuracy = 0
    for images, labels in test_loader:
        preds = torch.argmax(cnn_wi(images), axis=1, keepdim=False)
        accuracy += calc_metric(preds, labels)

    print("Accuracy for the test dataset: {:.3%}".format(accuracy / len(test_loader)))  

    if baseline_accuracy - accuracy > 0.1:
        seed_list.append(randint(1000, 5000))
        continue 

  0%|          | 0/10 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | SimpleCNNtorch   | 25.0 K | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | valid_metrics | MetricCollection | 0      | train
-----------------------------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)
25        Modules in train mode
0         Modules in eval mode


Epoch 12: 100%|██████████| 158/158 [00:03<00:00, 42.28it/s, v_num=nary, train_loss_step=7.32e-5, val_loss=0.0057, val_accuracy=0.998, val_auc=0.998, train_loss_epoch=0.006, train_accuracy=0.999, train_auc=0.999]    


 10%|█         | 1/10 [00:42<06:22, 42.47s/it]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | SimpleCNNtorch   | 25.0 K | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | valid_metrics | MetricCollection | 0      | train
-----------------------------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)
25        Modules in train mode
0         Modules in eval mode


Accuracy for the test dataset: 99.953%
Epoch 0: 100%|██████████| 158/158 [00:03<00:00, 45.17it/s, v_num=nary, train_loss_step=0.0158, val_loss=0.0194, val_accuracy=0.994, val_auc=0.993, train_loss_epoch=0.109]


 20%|██        | 2/10 [00:46<02:37, 19.68s/it]GPU available: False, used: False


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | SimpleCNNtorch   | 25.0 K | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | valid_metrics | MetricCollection | 0      | train
-----------------------------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)
25        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 158/158 [00:02<00:00, 55.46it/s, v_num=nary, train_loss_step=0.00337, val_loss=0.0253, val_accuracy=0.992, val_auc=0.991, train_loss_epoch=0.118]
Accuracy for the test dataset: 99.763%


 90%|█████████ | 9/10 [01:09<00:04,  4.01s/it]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | SimpleCNNtorch   | 25.0 K | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | valid_metrics | MetricCollection | 0      | train
-----------------------------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)
25        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 158/158 [00:03<00:00, 47.22it/s, v_num=nary, train_loss_step=0.0291, val_loss=0.0247, val_accuracy=0.991, val_auc=0.991, train_loss_epoch=0.136]
Accuracy for the test dataset: 99.669%


100%|██████████| 10/10 [01:13<00:00,  7.33s/it]
